In [ ]:
# !pip install pyautogui

In [1]:
import cv2
import mediapipe as mp
import math
import pyautogui




In [2]:
# MediaPipe hand detection module
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# OpenCV video capture
cap = cv2.VideoCapture(0)

# Create a hands object
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        ret, frame = cap.read()

        # Process the frame with MediaPipe
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Get the landmarks of the thumb and index finger
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                # Calculate the Euclidean distance between thumb and index finger
                distance = math.sqrt(
                    (thumb_tip.x - index_finger_tip.x) ** 2 + (thumb_tip.y - index_finger_tip.y) ** 2)

                # Map the distance to volume range (0-100)
                volume = int((distance * 100) / 0.4)  # Adjust the scaling factor as needed

                # Set the system volume
                pyautogui.press('volumedown') if volume < 50 else pyautogui.press('volumeup')

                # Display the volume level on the frame
                cv2.putText(frame, f"Volume: {volume}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the resulting frame
        cv2.imshow('Finger Volume Control', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Right hand only

In [7]:
# MediaPipe hand detection module
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# OpenCV video capture
cap = cv2.VideoCapture(0)
# Create a hands object
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)

        # Process the frame with MediaPipe
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Get the bounding box coordinates
                x_min = min(lm.x for lm in hand_landmarks.landmark)
                x_max = max(lm.x for lm in hand_landmarks.landmark)
                y_min = min(lm.y for lm in hand_landmarks.landmark)
                y_max = max(lm.y for lm in hand_landmarks.landmark)

                # Calculate the width and height of the bounding box
                width = x_max - x_min
                height = y_max - y_min

                # Consider only the right hand based on the bounding box
                if width > 0.3 and height > 0.3:

                    # Get the landmarks of the thumb and index finger
                    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                    # Calculate the Euclidean distance between thumb and index finger
                    distance = math.sqrt(
                        (thumb_tip.x - index_finger_tip.x) ** 2 + (thumb_tip.y - index_finger_tip.y) ** 2)

                    # Map the distance to volume range (0-100)
                    volume = int((distance * 100) / 0.3)  # Adjust the scaling factor as needed

                    # Set the system volume
                    pyautogui.press('volumedown') if volume < 50 else pyautogui.press('volumeup')

                    # Display the volume level on the frame
                    cv2.putText(frame, f"Volume: {volume}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

                    # Draw lines between fingers
                    cv2.line(frame, (int(thumb_tip.x * frame.shape[1]), int(thumb_tip.y * frame.shape[0])),
                             (int(index_finger_tip.x * frame.shape[1]), int(index_finger_tip.y * frame.shape[0])),
                             (0, 255, 0), 2)

                    # Draw hand landmarks on the frame
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the resulting frame
        cv2.imshow('Finger Volume Control', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


## Use libraly CTYPES

In [ ]:
# !pip install pycaw


In [ ]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel(  1)
# volume.GetVolumeRange( 0.0, 0.75)
# volume.SetMasterVolumeLevel(-20.0, None)

In [ ]:
def set_volume( volume_level):
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(
        IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    
    volume_range = volume.GetVolumeRange()
    min_volume = volume_range[0]
    max_volume = volume_range[1]

    # new_volume = max(0.0, volume_level - 0.1) 
    
    # Calculate the volume level within the range
    adjusted_volume = min_volume + (max_volume - min_volume) * volume_level
    
    # Set the adjusted volume level
    volume.SetMasterVolumeLevel(adjusted_volume, None)
    print("Volume : "+ str(volume.GetMasterVolumeLevelScalar()))

In [ ]:
def main():
    try:
        # devices = AudioUtilities.GetSpeakers()
        
        # Display available devices
        # for i, device in enumerate(devices):
        #     print(f"{i}: {device.FriendlyName}")
        
        # device_index = int(input("Enter the device index you want to control: "))
        volume_input = float(input("Enter the desired volume level (0.0 to 1.0): "))
        
        if 0 <= volume_input <= 1.0:
            # set_volume(devices[device_index], volume_input)
            set_volume( volume_input)

            print("Volume set successfully.")
        else:
            print("Invalid device index or volume level.")
    except Exception as e:
        print("An error occurred:", str(e))

if __name__ == "__main__":
    main()

## Volume function to percent input

In [2]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

def SetVolumePercent(percent_input):

            # percent_input = 50
    min_range_volume = 0.7
    max_range_volume = 1.0
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(   IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    if(percent_input != 0):


        volume_range = volume.GetVolumeRange()
        min_volume = volume_range[0]
        max_volume = volume_range[1]

        # print(volume_range)

        scaled_result = min_range_volume + (max_range_volume - min_range_volume) * (percent_input / 100.0)
        # print(scaled_result)
        adjusted_volume = min_volume + (max_volume - min_volume) * scaled_result
        # print(adjusted_volume)
        volume.SetMasterVolumeLevel(adjusted_volume, None)
        # print("Volume : "+ str(volume.GetMasterVolumeLevelScalar()))
    else:
        volume.SetMasterVolumeLevel(-63, None)
        # print("Volume : "+ str(volume.GetMasterVolumeLevelScalar()))



    return True

In [9]:
# SetVolumePercent(50)

True

## Hand control Up volume number

In [1]:
# MediaPipe hand detection module
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# OpenCV video capture
cap = cv2.VideoCapture(0)
# Create a hands object
with mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally
        frame = cv2.flip(frame, 1)

        # Process the frame with MediaPipe
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Get the bounding box coordinates
                x_min = min(lm.x for lm in hand_landmarks.landmark)
                x_max = max(lm.x for lm in hand_landmarks.landmark)
                y_min = min(lm.y for lm in hand_landmarks.landmark)
                y_max = max(lm.y for lm in hand_landmarks.landmark)

                # Calculate the width and height of the bounding box
                width = x_max - x_min
                height = y_max - y_min

                # Consider only the right hand based on the bounding box
                if width > 0.3 and height > 0.3:

                    # Get the landmarks of the thumb and index finger
                    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

                    # Calculate the Euclidean distance between thumb and index finger
                    distance = math.sqrt(
                        (thumb_tip.x - index_finger_tip.x) ** 2 + (thumb_tip.y - index_finger_tip.y) ** 2)

                    # Map the distance to volume range (0-100)
                    volume = int((distance * 100) / 0.3)  # Adjust the scaling factor as needed

                    # Set the system volume
                    SetVolumePercent(volume)

                    # Display the volume level on the frame
                    cv2.putText(frame, f"Volume: {volume}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

                    # Draw lines between fingers
                    cv2.line(frame, (int(thumb_tip.x * frame.shape[1]), int(thumb_tip.y * frame.shape[0])),
                             (int(index_finger_tip.x * frame.shape[1]), int(index_finger_tip.y * frame.shape[0])),
                             (0, 255, 0), 2)

                    # Draw hand landmarks on the frame
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display the resulting frame
        cv2.imshow('Finger Volume Control', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

NameError: name 'mp' is not defined